In [1]:
# Import Libraries
import os
import dgl
import time
import torch
import optuna
import datetime
import warnings
import numpy as np
from torch import nn
from tqdm import tqdm
import torch.nn as nn
from dgl.nn import GATConv
import torch.optim as optim
from collections import Counter
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, hamming_loss
from sklearn.metrics import confusion_matrix, cohen_kappa_score, roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
# The purpose is process and organize batched data, ensuring that it can be efficiently used in the model
def custom_collate_fn(batch):
    graphs, d1_sim, d2_embeddings, d2_sim, labels, indices = zip(*batch)

    # Batch graphs
    batched_graphs = dgl.batch(graphs)

    # Convert tensors
    d1_sim = torch.stack(d1_sim)  # Small molecule similarity
    d2_embeddings = torch.stack(d2_embeddings)  # Biotech embeddings
    d2_sim = torch.stack(d2_sim)  # Biotech similarity
    labels = torch.tensor(labels, dtype=torch.long)

    # Convert indices to tensor for the batch
    indices = torch.tensor(indices, dtype=torch.long)

    return batched_graphs, d1_sim, d2_embeddings, d2_sim, labels, indices


In [3]:
def parse_scientific_notation(value):
    """
    Converts a value from scientific notation (or other numeric formats) to an integer.

    Args:
        value (str): Input value as a string.

    Returns:
        int or None: The integer representation of the value, or None if conversion fails.
    """
    try:
        return int(float(value))
    except ValueError:
        # Log a warning if the value cannot be converted
        warnings.warn(f"Could not convert value: {value}. Returning None.")
        return None


In [4]:
def read_label_file(file_path):
    """
    Reads a file containing labels and converts them into integers.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines = [parse_scientific_notation(line.strip()) for line in lines]
    return lines

def read_combinations_index_file(file_path):
    """
    Reads a file containing index pairs (biotech_idx, small_molecule_idx).
    """
    combinations = []
    with open(file_path, 'r') as file:
        for line in file:
            biotech_idx, small_molecule_idx = map(int, line.strip().split())
            combinations.append((biotech_idx, small_molecule_idx))
    return combinations

# Load data files
data_path = "../Data/chemical_bi-directed_dgl_graphs_and_labels.pt"
label_file_path = "../Data/31_Y_ratio1.txt"
combinations_file_path = "../Data/31_XIndex_ratio1.txt"

try:
    data = torch.load(data_path)
    small_molecule_graphs, small_molecule_labels = data
except FileNotFoundError:
    raise FileNotFoundError(f"File not found: {data_path}")

# Process small molecule features
small_molecule_features = []
for g in small_molecule_graphs:
    if 'atomic' in g.ndata:  # Node features already exist
        feature = g.ndata['atomic']
    else:
        warnings.warn(f"Graph {g} is missing 'atomic' features. Assigning default ones.")
        feature = torch.ones(g.num_nodes(), 1)  # Default features
    small_molecule_features.append(feature)

# Load matrices and labels
small_molecule_total_matrix = torch.tensor(
    np.loadtxt('../Data/total_small_drugs.txt'), dtype=torch.float
)
biotech_embeddings = torch.tensor(
    np.loadtxt('../Data/mean_sequence_embeddings_Protbert2.txt'), dtype=torch.float
)
biotech_total_matrix = torch.tensor(
    np.loadtxt('../Data/total_biotech_drugs.txt'), dtype=torch.float
)
labels_list = read_label_file(label_file_path)
combinations = read_combinations_index_file(combinations_file_path)

# Create the InteractionDataset class
class InteractionDataset(Dataset):
    def __init__(self, small_molecule_graphs, small_molecule_total_matrix,
                 biotech_embeddings, biotech_total_matrix, labels, combinations):
        """
        Dataset for drug-drug interactions with precomputed indices.
        """
        self.small_molecule_graphs = small_molecule_graphs
        self.small_molecule_total_matrix = small_molecule_total_matrix
        self.biotech_embeddings = biotech_embeddings
        self.biotech_total_matrix = biotech_total_matrix
        self.labels = labels
        self.combinations = combinations

    def __len__(self):
        return len(self.combinations)

    def __getitem__(self, idx):
        biotech_idx, small_idx = self.combinations[idx]
        d1_graph = self.small_molecule_graphs[small_idx]  # DGL graph
        d1_sim = self.small_molecule_total_matrix[small_idx]  # Similarity matrix row
        d2_embedding = self.biotech_embeddings[biotech_idx]  # ProtBERT embeddings
        d2_sim = self.biotech_total_matrix[biotech_idx]  # Similarity matrix row
        label = self.labels[idx]  # Label for the interaction

        # Check dimensions
        if d1_sim.shape != (2148,):
            warnings.warn(f"d1_sim shape mismatch: {d1_sim.shape}")
        if d2_embedding.shape != (1024,):
            warnings.warn(f"d2_embedding shape mismatch: {d2_embedding.shape}")
        if d2_sim.shape != (196,):
            warnings.warn(f"d2_sim shape mismatch: {d2_sim.shape}")

        return d1_graph, d1_sim, d2_embedding, d2_sim, label, (biotech_idx, small_idx)

# Create the dataset
labels = torch.tensor(labels_list, dtype=torch.long)  # Convert labels list to tensor
dataset = InteractionDataset(
    small_molecule_graphs=small_molecule_graphs,         # List of small molecule graphs
    small_molecule_total_matrix=small_molecule_total_matrix,  # Total similarity matrix for small molecules
    biotech_embeddings=biotech_embeddings,              # Biotech ProtBERT embeddings
    biotech_total_matrix=biotech_total_matrix,          # Total similarity matrix for biotech drugs
    labels=labels,                                # Labels tensor
    combinations=combinations                           # Precomputed (biotech_idx, small_idx) pairs
)

# Debugging outputs
print(f"Dataset size: {len(dataset)}")
print(f"Number of small molecule graphs: {len(small_molecule_graphs)}")
print(f"Small molecule total matrix shape: {small_molecule_total_matrix.shape}")
print(f"Biotech embeddings shape: {biotech_embeddings.shape}")
print(f"Biotech total matrix shape: {biotech_total_matrix.shape}")
print(f"Number of unique labels: {len(torch.unique(labels))}")


Dataset size: 87752
Number of small molecule graphs: 2148
Small molecule total matrix shape: torch.Size([2148, 2148])
Biotech embeddings shape: torch.Size([196, 1024])
Biotech total matrix shape: torch.Size([196, 196])
Number of unique labels: 32


In [5]:
def create_sampler(dataset, indices, labels):
    """
    Create a WeightedRandomSampler to handle class imbalance in the dataset.

    Args:
        dataset (Dataset): The full dataset.
        indices (list): Indices for the subset of the dataset to sample from.
        labels (Tensor): Full label tensor for the dataset.

    Returns:
        WeightedRandomSampler: Sampler for DataLoader.
    """
    subset_labels = [labels[i].item() for i in indices]
    class_counts = Counter(subset_labels)
    sample_weights = [1.0 / class_counts[label] for label in subset_labels]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(indices), replacement=True)
    return sampler

def get_class_distribution(dataset, indices):
    """
    Get the class distribution in a dataset subset.

    Args:
        dataset (Dataset): The full dataset.
        indices (list): Indices for the subset.

    Returns:
        dict: Dictionary of class distributions {class_label: count}.
    """
    subset_labels = [dataset[i][4].item() for i in indices]
    unique, counts = np.unique(subset_labels, return_counts=True)
    return dict(zip(unique, counts))

# Define dataset indices
indices = list(range(len(dataset)))  # List of indices for all samples


In [6]:
# Generate full labels for the entire dataset
full_labels = [dataset[i][4].item() for i in range(len(dataset))]
full_labels = torch.tensor(full_labels, dtype=torch.long)  # Ensure correct tensor type
print(f"Full labels tensor size: {full_labels.shape} (Expected: [dataset size])")

# Generate indices for the entire dataset
indices = list(range(len(dataset)))


Full labels tensor size: torch.Size([87752]) (Expected: [dataset size])


In [7]:
num_classes=32
class GATModel(nn.Module):
    def __init__(self, in_dim, hidden_dims, out_dim, num_heads, dropout=0.2):
        super(GATModel, self).__init__()
        self.gat_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()  # List of LayerNorm layers
        self.dropout = nn.Dropout(dropout)

        prev_dim = in_dim
        for hidden_dim in hidden_dims:
            self.gat_layers.append(
                dgl.nn.GATConv(
                    in_feats=prev_dim,
                    out_feats=hidden_dim,
                    num_heads=num_heads,
                    allow_zero_in_degree=True,
                )
            )
            self.batch_norms.append(nn.LayerNorm(hidden_dim * num_heads))  # Use LayerNorm
            prev_dim = hidden_dim * num_heads

        # Final fully connected layer
        self.fc = nn.Linear(prev_dim, out_dim)

    def forward(self, g, h):
        for gat_layer, norm in zip(self.gat_layers, self.batch_norms):
            h = self.dropout(h)
            h = torch.relu(gat_layer(g, h).flatten(1))
            h = norm(h)  # Apply LayerNorm
        return self.fc(h)


class MultiClassMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.2):
        super(MultiClassMLP, self).__init__()
        layers = []
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.LayerNorm(hidden_dim))  # Apply LayerNorm first
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = hidden_dim
        layers.append(nn.Linear(input_dim, num_classes))  # Final layer
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.fc_layers(x)


class CombinedModel(nn.Module):
    def __init__(self, gat_input_dim, gat_hidden_dims, gat_output_dim,
                 mlp_hidden_dims, num_classes, sim_dim_small, sim_dim_biotech, dropout=0.2):
        super(CombinedModel, self).__init__()
        self.gat = GATModel(
            in_dim=gat_input_dim,
            hidden_dims=gat_hidden_dims,
            out_dim=gat_output_dim,
            num_heads=4,  # Number of attention heads
            dropout=dropout,
        )

        self.mlp_input_dim = gat_output_dim + sim_dim_small + 1024 + sim_dim_biotech
        self.mlp = MultiClassMLP(self.mlp_input_dim, mlp_hidden_dims, num_classes, dropout)

    def forward(self, graphs, atomic_features, batch_drug1_sim, batch_drug2_tensor, batch_drug2_sim):
        # Process small molecule graphs through GAT
        node_embeddings = self.gat(graphs, atomic_features)
        graphs.ndata['h'] = node_embeddings
        graph_embeddings = dgl.mean_nodes(graphs, 'h')  # Graph-level embeddings for the batch

        # Concatenate inputs for MLP
        drug1_input = torch.cat([batch_drug2_tensor, batch_drug1_sim], dim=1)
        drug2_input = torch.cat([graph_embeddings, batch_drug2_sim], dim=1)
        combined_input = torch.cat([drug1_input, drug2_input], dim=1)

        return self.mlp(combined_input)


In [8]:
import os
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    matthews_corrcoef,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
)
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Helper function to ensure directory exists
def ensure_dir_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


# Save results for each fold and averages for all formats
def save_results_to_files(config_id, fold_metrics, avg_metrics, output_dir):
    ensure_dir_exists(output_dir)
    formats = ["Micro", "Macro", "Weighted"]

    for metric_format in formats:
        file_path = os.path.join(output_dir, f"{config_id}_{metric_format}_results.txt")
        with open(file_path, "w") as file:
            file.write(f"{metric_format} Metrics for Config: {config_id}\n")
            
            # Save fold results
            if metric_format in fold_metrics:
                for fold, metrics in enumerate(fold_metrics[metric_format], start=1):
                    file.write(f"\nFold {fold}:\n")
                    for metric, value in metrics.items():
                        if metric.endswith("STD"):
                            file.write(f"  {metric}: ±{value:.4f}\n")
                        else:
                            file.write(f"  {metric}: {value:.4f}\n")
            
            # Save average metrics
            if avg_metrics:
                file.write("\nAverage Metrics Across Folds:\n")
                for metric, value in avg_metrics[metric_format].items():
                    if metric.endswith("STD"):
                        file.write(f"  {metric}: ±{value:.4f}\n")
                    else:
                        file.write(f"  {metric}: {value:.4f}\n")

    #print(f"[INFO] Results saved for all formats in {output_dir}")


# Training function
def train(model, loader, optimizer, criterion, device, epoch, config_id, fold):
    model.train()
    total_loss = 0
    train_progress = tqdm(loader, desc=f"[{config_id}] Fold {fold} | Epoch {epoch} Training", unit="batch")

    for batch_idx, batch in enumerate(train_progress):
        graphs, d1_sim, d2_embeddings, d2_sim, labels = (
            batch[0].to(device), batch[1].to(device), batch[2].to(device),
            batch[3].to(device), batch[4].to(device)
        )

        optimizer.zero_grad()
        outputs = model(graphs, graphs.ndata['atomic'], d1_sim, d2_embeddings, d2_sim)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        train_progress.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    #print(f"[INFO] [{config_id}] Fold {fold} | Epoch {epoch} Avg Loss: {avg_loss:.4f}")
    return avg_loss

def evaluate(model, loader, device, fold, config_id, num_classes):
    preds, true_labels, prob_outputs, _, _ = evaluate_and_return_preds(model, loader, device)

    metrics = {"Micro": {}, "Macro": {}, "Weighted": {}}

    # Micro metrics
    metrics["Micro"]["Accuracy"] = accuracy_score(true_labels, preds)
    metrics["Micro"]["Precision"], metrics["Micro"]["Recall"], metrics["Micro"]["F1"], _ = \
        precision_recall_fscore_support(true_labels, preds, average='micro')
    metrics["Micro"]["MCC"] = matthews_corrcoef(true_labels, preds)
    metrics["Micro"]["AUC"] = roc_auc_score(true_labels, prob_outputs, average='micro', multi_class='ovr')
    metrics["Micro"]["AUPR"] = np.mean([
        average_precision_score((np.array(true_labels) == i).astype(int), np.array(prob_outputs)[:, i])
        for i in range(num_classes)
    ])

    # Macro metrics
    precision_per_class, recall_per_class, f1_per_class, _ = \
        precision_recall_fscore_support(true_labels, preds, average=None, zero_division=0)
    auc_per_class = [
        roc_auc_score((np.array(true_labels) == i).astype(int), np.array(prob_outputs)[:, i])
        for i in range(num_classes)
    ]
    aupr_per_class = [
        average_precision_score((np.array(true_labels) == i).astype(int), np.array(prob_outputs)[:, i])
        for i in range(num_classes)
    ]
    metrics["Macro"]["Precision"] = np.mean(precision_per_class)
    metrics["Macro"]["Recall"] = np.mean(recall_per_class)
    metrics["Macro"]["F1"] = np.mean(f1_per_class)
    metrics["Macro"]["Precision_STD"] = np.std(precision_per_class)
    metrics["Macro"]["Recall_STD"] = np.std(recall_per_class)
    metrics["Macro"]["F1_STD"] = np.std(f1_per_class)
    metrics["Macro"]["AUC"] = np.mean(auc_per_class)
    metrics["Macro"]["AUPR"] = np.mean(aupr_per_class)
    metrics["Macro"]["AUC_STD"] = np.std(auc_per_class)
    metrics["Macro"]["AUPR_STD"] = np.std(aupr_per_class)

    # Weighted metrics
    class_weights = np.bincount(true_labels) / len(true_labels)
    metrics["Weighted"]["Precision"] = np.sum(precision_per_class * class_weights)
    metrics["Weighted"]["Recall"] = np.sum(recall_per_class * class_weights)
    metrics["Weighted"]["F1"] = np.sum(f1_per_class * class_weights)
    metrics["Weighted"]["Precision_STD"] = np.sqrt(
        np.sum((precision_per_class - metrics["Weighted"]["Precision"]) ** 2 * class_weights)
    )
    metrics["Weighted"]["Recall_STD"] = np.sqrt(
        np.sum((recall_per_class - metrics["Weighted"]["Recall"]) ** 2 * class_weights)
    )
    metrics["Weighted"]["F1_STD"] = np.sqrt(
        np.sum((f1_per_class - metrics["Weighted"]["F1"]) ** 2 * class_weights)
    )
    metrics["Weighted"]["AUC"] = np.sum(np.array(auc_per_class) * class_weights)
    metrics["Weighted"]["AUPR"] = np.sum(np.array(aupr_per_class) * class_weights)
    metrics["Weighted"]["AUC_STD"] = np.sqrt(
        np.sum((np.array(auc_per_class) - metrics["Weighted"]["AUC"]) ** 2 * class_weights)
    )
    metrics["Weighted"]["AUPR_STD"] = np.sqrt(
        np.sum((np.array(aupr_per_class) - metrics["Weighted"]["AUPR"]) ** 2 * class_weights)
    )

    # Save confusion matrix
    cm = confusion_matrix(true_labels, preds, labels=np.arange(num_classes))
    cm_dir = "confusion_matrices"
    ensure_dir_exists(cm_dir)
    cm_file = os.path.join(cm_dir, f"{config_id}_fold_{fold}_confusion_matrix.txt")
    np.savetxt(cm_file, cm, fmt='%d', delimiter='\t')
  #  print(f"[INFO] Confusion Matrix for Fold {fold} saved to {cm_file}")

    return metrics




def evaluate_and_return_preds(model, loader, device):
    model.eval()
    preds, true_labels, prob_outputs = [], [], []
    biotech_indices, small_molecule_indices = [], []

    with torch.no_grad():
        # Initialize progress bar
        eval_progress = tqdm(loader, desc="Evaluating", unit="batch")

        for batch in eval_progress:
            graphs, d1_sim, d2_embeddings, d2_sim, labels, batch_indices = (
                batch[0].to(device), batch[1].to(device), batch[2].to(device),
                batch[3].to(device), batch[4].to(device), batch[5]
            )
            outputs = model(graphs, graphs.ndata['atomic'], d1_sim, d2_embeddings, d2_sim)
            probabilities = torch.softmax(outputs, dim=1)
            predictions = torch.argmax(outputs, dim=1)

            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            prob_outputs.extend(probabilities.cpu().numpy())
            
            # Unpack batch indices into biotech and small molecule indices
            for biotech_idx, small_molecule_idx in batch_indices:
                biotech_indices.append(biotech_idx)
                small_molecule_indices.append(small_molecule_idx)

    return preds, true_labels, prob_outputs, biotech_indices, small_molecule_indices



# Save final predictions across all folds
def save_final_predictions(config_id, final_preds, final_true_labels, final_biotech_indices, final_small_molecule_indices, save_dir):
    ensure_dir_exists(save_dir)
    final_predictions_file = os.path.join(save_dir, f"{config_id}_final_predictions.txt")
    with open(final_predictions_file, "w") as file:
        file.write("Biotech_Index\tSmall_Molecule_Index\tTrue_Label\tPredicted_Label\n")
        for biotech_idx, small_idx, true, pred in zip(final_biotech_indices, final_small_molecule_indices, final_true_labels, final_preds):
            file.write(f"{biotech_idx}\t{small_idx}\t{true}\t{pred}\n")

    print(f"[INFO] Final predictions saved to {final_predictions_file}")


# Cross-validation function
def cross_validate(model, dataset, full_labels, num_splits, batch_size, device, config, config_id, save_dir):
    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    all_metrics = {"Micro": [], "Macro": [], "Weighted": []}

    # To aggregate predictions and indices across folds
    final_preds, final_true_labels = [], []
    final_biotech_indices, final_small_molecule_indices = [], []

    for fold, (train_indices, val_indices) in enumerate(skf.split(range(len(dataset)), full_labels.numpy())):
        print(f"\n[INFO] [{config_id}] Starting Fold {fold + 1}/{num_splits}")
        train_set = Subset(dataset, train_indices)
        val_set = Subset(dataset, val_indices)

        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
        criterion = torch.nn.CrossEntropyLoss()

        # Training
        for epoch in range(10):  # Adjust epochs as needed
            train(model, train_loader, optimizer, criterion, device, epoch, config_id, fold + 1)

        # Evaluation
        preds, true_labels, prob_outputs, biotech_indices, small_molecule_indices = evaluate_and_return_preds(
            model, val_loader, device
        )

        # Aggregate predictions and indices
        final_preds.extend(preds)
        final_true_labels.extend(true_labels)
        final_biotech_indices.extend(biotech_indices)
        final_small_molecule_indices.extend(small_molecule_indices)

        # Compute metrics
        metrics = evaluate(model, val_loader, device, fold + 1, config_id, num_classes=32)

        # Store metrics for this fold
        for fmt in ["Micro", "Macro", "Weighted"]:
            format_metrics = {metric: metrics[fmt][metric] for metric in metrics[fmt]}
            all_metrics[fmt].append(format_metrics)

        # Save fold results immediately
        fold_results_dir = os.path.join(save_dir, f"fold_{fold + 1}")
        ensure_dir_exists(fold_results_dir)
        save_results_to_files(config_id, {fmt: [metrics[fmt]] for fmt in metrics}, {}, fold_results_dir)

    # Compute average metrics across folds
    avg_metrics = {}
    for fmt in ["Micro", "Macro", "Weighted"]:
        avg_metrics[fmt] = {}
        for metric in all_metrics[fmt][0].keys():
            values = [fold_metrics[metric] for fold_metrics in all_metrics[fmt]]
            avg_metrics[fmt][metric] = np.mean(values)
            if not metric.endswith("STD"):
                avg_metrics[fmt][f"{metric}_STD"] = np.std(values)

    # Save overall average metrics
    avg_results_dir = os.path.join(save_dir, "averages")
    ensure_dir_exists(avg_results_dir)
    save_results_to_files(config_id, all_metrics, avg_metrics, avg_results_dir)

    # Save predictions after all folds
    save_final_predictions(
        config_id=config_id,
        final_preds=final_preds,
        final_true_labels=final_true_labels,
        final_biotech_indices=final_biotech_indices,
        final_small_molecule_indices=final_small_molecule_indices,
        save_dir=save_dir,
    )

    return avg_metrics

    # Save overall average metrics
    avg_results_dir = os.path.join(save_dir, "averages")
    ensure_dir_exists(avg_results_dir)
    save_results_to_files(config_id, all_metrics, avg_metrics, avg_results_dir)

    return avg_metrics



# Hyperparameter grid search
def run_all_combinations(hyperparameters, dataset, full_labels, device, save_dir):
    for gat_hidden_dims in hyperparameters["gat_hidden_dims"]:
        for mlp_hidden_dims in hyperparameters["mlp_hidden_dims"]:
            for dropout in hyperparameters["dropout"]:
                for learning_rate in hyperparameters["learning_rate"]:
                    for batch_size in hyperparameters["batch_size"]:
                        config = {
                            "gat_hidden_dims": gat_hidden_dims,
                            "mlp_hidden_dims": mlp_hidden_dims,
                            "dropout": dropout,
                            "learning_rate": learning_rate,
                            "batch_size": batch_size,
                        }
                        config_id = f"GAT_{'_'.join(map(str, gat_hidden_dims))}_MLP_{'_'.join(map(str, mlp_hidden_dims))}_LR_{learning_rate:.1e}_BS_{batch_size}_DO_{dropout}"
                        print(f"\n[INFO] Starting Config: {config_id}")

                        model = CombinedModel(
                            gat_input_dim=1,
                            gat_hidden_dims=gat_hidden_dims,
                            gat_output_dim=512,
                            mlp_hidden_dims=mlp_hidden_dims,
                            num_classes=num_classes,
                            sim_dim_small=2148,
                            sim_dim_biotech=196,
                            dropout=dropout,
                        ).to(device)

                        cross_validate(model, dataset, full_labels, num_splits=10, batch_size=batch_size, device=device, config=config, config_id=config_id, save_dir=save_dir)

# Define hyperparameters
hyperparameters = {
    "gat_hidden_dims": [[128, 256]],
    "mlp_hidden_dims": [[1024, 512]], #[[256], [256, 512], [256, 512, 1024]]
    "dropout": [0.2],
    "learning_rate": [1e-3],
    "batch_size": [32],
}

# Run grid search
run_all_combinations(hyperparameters, dataset, full_labels, device=device, save_dir="results")




[INFO] Starting Config: GAT_128_256_MLP_1024_512_LR_1.0e-03_BS_32_DO_0.2

[INFO] [GAT_128_256_MLP_1024_512_LR_1.0e-03_BS_32_DO_0.2] Starting Fold 1/10


[GAT_128_256_MLP_1024_512_LR_1.0e-03_BS_32_DO_0.2] Fold 1 | Epoch 0 Training:  14%|▏| 341/2468 [01:07<07:02,  5.03batch


KeyboardInterrupt: 